Imports

In [1]:
import numpy as np
import pandas as pd
import datetime

import pandas_datareader # install with `pip install pandas-datareader`
from dstapi import DstApi # install with `pip install git+https://github.com/alemartinello/dstapi`

import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

We import our data from Statistics Denmark (consumer price index and gross unemployment rate):

In [8]:
CPI = DstApi('PRIS111') 
GUR = DstApi('AUS07') 

A quick overview of the available data:

In [11]:
tabsum1 = CPI.tablesummary(language='en')
display(tabsum1)
tabsum2 = GUR.tablesummary(language='en')
display(tabsum2)

Table PRIS111: Consumer price index by commodity group, unit and time
Last update: 2023-04-11T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,VAREGR,384,000000,"00 Consumer price index, total",151000,15.1 Overall Consumer Price Index excl. energy,False
1,ENHED,3,100,Index,300,Percentage change compared to same month the y...,False
2,Tid,267,2001M01,2001M01,2023M03,2023M03,True


Table AUS07: Unemployed persons (seasonally adjusted) by benefits, sex and age, seasonal adjustment and actual figures and time
Last update: 2023-03-30T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,YD,15,TOT,Gross unemployment,O60,60 years and more,False
1,SAESONFAK,4,9,Seasonally adjusted figures in percent of the ...,24,Enumerated actual figures,False
2,Tid,194,2007M01,2007M01,2023M02,2023M02,True


In [13]:
# The available values for a each variable in data from CPI: 
for variable in tabsum1['variable name']:
    print(variable+':')
    display(CPI.variable_levels(variable, language='en'))

VAREGR:


,id,text
0,000000,"00 Consumer price index, total"
1,010000,01 Food and non-alcoholic beverages
2,011000,01.1 Food
3,011100,01.1.1 Bread and cereals
4,011110,01.1.1.1 Rice
...,...,...
379,127030,12.7.0.3 Funeral services
380,131000,13.1 Goods (total)
381,132000,13.2 Services (total)
382,141000,14.1 Overall Consumer Price Index - excl. ener...


ENHED:


,id,text
0,100,Index
1,200,Percentage change compared to previous month (...
2,300,Percentage change compared to same month the y...


Tid:


,id,text
0,2001M01,2001M01
1,2001M02,2001M02
2,2001M03,2001M03
3,2001M04,2001M04
4,2001M05,2001M05
...,...,...
262,2022M11,2022M11
263,2022M12,2022M12
264,2023M01,2023M01
265,2023M02,2023M02


In [15]:
# The available values for a each variable in data from GUR: 
for variable in tabsum2['variable name']:
    print(variable+':')
    display(GUR.variable_levels(variable, language='en'))

YD:


,id,text
0,TOT,Gross unemployment
1,NET,Net unemployment
2,LDP,Net unemployed recipients of unemployment bene...
3,LKT,Net unemployed recipients of social assistance
4,AKI,Activation total
5,ADP,Activation of persons on unemployment benefits
6,AKT,Activation of persons on social assistance (pr...
7,MEN,Men
8,KVR,Women
9,U25,16-24 years


SAESONFAK:


,id,text
0,9,Seasonally adjusted figures in percent of the ...
1,10,Seasonally adjusted
2,22,Enumerated actual figures in percent of the l...
3,24,Enumerated actual figures


Tid:


,id,text
0,2007M01,2007M01
1,2007M02,2007M02
2,2007M03,2007M03
3,2007M04,2007M04
4,2007M05,2007M05
...,...,...
189,2022M10,2022M10
190,2022M11,2022M11
191,2022M12,2022M12
192,2023M01,2023M01
